In [1]:
'''Sequence to sequence example in Keras (character-level).

This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

# Summary of the algorithm

- We start with input sequences from a domain (e.g. English sentences)
    and correspding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    Is uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.

# Data download

English to French sentence pairs.
http://www.manythings.org/anki/fra-eng.zip

Lots of neat sentence pairs datasets can be found at:
http://www.manythings.org/anki/

# References

- Sequence to Sequence Learning with Neural Networks
    https://arxiv.org/abs/1409.3215
- Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    https://arxiv.org/abs/1406.1078
'''
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = '.keras/datasets/fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path).read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#
# data preparation
#
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Using TensorFlow backend.


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59


In [3]:
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print (model.summary())


# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states


print ('+++ encoder_model: ')

encoder_model = Model(encoder_inputs, encoder_states)

print (encoder_model.summary())

#   
#    Decoder model for inference

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

print ('+++ decoder model: ')
print (decoder_model.summary())

dec_in = [decoder_inputs] + decoder_states_inputs
dec_out = [decoder_outputs] + decoder_states
print (dec_in)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, 71)      0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, None, 93)      0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, 256), (None,  335872      input_2[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    [(None, None, 256), ( 358400      input_3[0][0]                    
                                                                   lstm_1[0][1]            

In [7]:
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.


# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
modelfile = 's2s.h5'
model.save(modelfile)
print ('model saved to ', modelfile)


Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 17s - loss: 0.7299 - val_loss: 0.7798

In [5]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [8]:
# sample run

for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
#EOF

-
Input sentence: Go.
Decoded sentence: Attends une bous !

-
Input sentence: Run!
Decoded sentence: Arrête de le porte !

-
Input sentence: Run!
Decoded sentence: Arrête de le porte !

-
Input sentence: Wow!
Decoded sentence: Attends une bous !

-
Input sentence: Fire!
Decoded sentence: Attendez un coup !

-
Input sentence: Help!
Decoded sentence: Prenez une parte !

-
Input sentence: Jump.
Decoded sentence: Attends une bous !

-
Input sentence: Stop!
Decoded sentence: Arrête de le porte !

-
Input sentence: Stop!
Decoded sentence: Arrête de le porte !

-
Input sentence: Stop!
Decoded sentence: Arrête de le porte !

-
Input sentence: Wait!
Decoded sentence: Attendez une coute !

-
Input sentence: Wait!
Decoded sentence: Attendez une coute !

-
Input sentence: I see.
Decoded sentence: Je l'ai vuille.

-
Input sentence: I try.
Decoded sentence: Je l'ai pas éci.

-
Input sentence: I won!
Decoded sentence: J'aime le reux.

-
Input sentence: I won!
Decoded sentence: J'aime le reux.

-
Inpu

In [37]:
s = u'한국말'
print ('s = ', s)
print ('s[0]= ', s[0])

a = s.encode('utf-8')
print ('encoded: ', a)
print (a[0], a[1], 'len(a) = ', len(a))
#for n in a:
#    print (n)
    
print ('decoded: ', a.decode('utf-8'))
b = chr(a[0]) + chr(a[1]) + chr(a[2])
c = b.encode('utf-8') #decode('utf-8')
print (b)
print (c)

s =  한국말
s[0]=  한
encoded:  b'\xed\x95\x9c\xea\xb5\xad\xeb\xa7\x90'
237 149 len(a) =  9
decoded:  한국말
í
b'\xc3\xad\xc2\x95\xc2\x9c'


In [42]:
aa = u'한'
bb = aa.encode('utf-8')
#cc = unicode(aa)
print (aa, bb, 'bb[0]: ', bb[0])

한 b'\xed\x95\x9c' bb[0]:  237


### 기본적으로 윈도우 환경의 Python은 ANSI 형식을 따르기 때문에 한글이 깨지는 경우가 발생합니다. 그래서 기본 인코딩 형식을 utf-8로 바꾸셔야 텍스트를 불러왔을때 깨지지 않게 됩니다.
#### http://www.lifebloom.biz/2017/07/04/python을-활용한-텍스트-마이닝-2-텍스트-분석-데이터-전처/

In [45]:
#-*- coding: utf-8 -*-

from urllib.request import urlopen
from bs4 import BeautifulSoup


In [47]:
url = 'http://movie.daum.net/moviedb/grade?movieId=97728&type=netizen&page=2'

In [52]:
webpage=urlopen(url)

source = BeautifulSoup(webpage,'html.parser',from_encoding='utf-8')
print (source)


<!DOCTYPE html>

<html class="os_unknown none unknown version_0" lang="ko">
<head>
<meta charset="utf-8">
<meta content="Daum 영화" property="og:site_name">
<meta content="IE=edge" http-equiv="X-UA-Compatible">
<!-- <title>$tagfilter.filter($stringCutUtil.cutStringRealLength($tmp, $len, "..")) - Daum 영화</title> -->
<title>Daum 영화</title>
<meta content="Daum영화 &lt;옥자&gt;" property="og:title">
<meta content="http://img1.daumcdn.net/thumb/C300x200/?fname=http%3A%2F%2Ft1.daumcdn.net%2Fmovie%2Fca5d4a03d36184dbfa7c3101745b5cac37aea30c" property="og:image">
<meta content="우린 집으로 갈거야, 반드시 함께강원도 산골 소녀 ‘미자’(안서현)에게 옥자는 10년 간 함께 자란 둘도 없는 친구이자 소중한 가족이다. 자연 속에서 평화롭게 지내던 어느 날, 글로벌 기업 ‘미란도’가 나타나 갑자기 옥자를 뉴욕으로 끌고가고, 할아버지(변희봉)의 만류에도 미자는 무작정 옥자를 구하기 위해 위험천만한 여정에 나선.." property="og:description">
<meta content="video.movie" property="og:type">
<meta content="" name="plink"/>
<meta content="" property="dg:plink"/>
<meta content="" property="article:mobile_view_url"/> <!-- mobile -->
<meta content="" property=

In [53]:
reviews = source.findAll('p',{'class': 'desc_review'})

for review in reviews:
    print(review.get_text().strip())

어디선가 벌어지고 있을지도 모르는 일이란 생각이 든다. 육실을 즐기지는 않지만...동물을 소비하기 위해 우린 얼마나 많은 것을 잃어야 하는지 생각게 한다.
잔혹동화
육식에 대한 불편함은 분명 고민해야 할 부분인거 같다. 지금의 사육 환경을 본다면 이런 영화들이 주는 메세지가 결코 가볍게 다가오지만은 않는다. 다만 영화가 주는 메세지에 비해 연출은 다소 힘이 빠진다.
10점 주는데도 참 고민이 많았던 영화다. 그만큼  생각하게 되고 어려운 주제가 아닐까 싶다. 특히 마지막 장면에서는 고기에 대한 불편한 시각을 줄수 있어 심히 우려가 되는 부분이다. 왜냐하면 인류를 생각하면 필이 없어서는 안될 중요한 자원이 되기 때문이다. 그래서 영화 자체만으로 봐서 10점을 주기로 했다.
감독이 채식주의자냐 ...  양심껏 일하고있는 축산업계 사람들과 .. 잘먹고있는 사람들에게  왜 고기에대한 혐오감을 갖게하는지 모르겠네.. 재미도 하나없고 이게 뭔 영화인지 ..
예쁘고 현실적인 동화. 그래서 옥자와 미자는 산 속에서 행복하게 살았을까?


배우 안서현이 연기를 잘해서 좋았다.



In [56]:
s = reviews[0].get_text().strip()
print (s[0])

어


#### Meaning of data type bytes in python
#### https://stackoverflow.com/questions/6269765/what-does-the-b-character-do-in-front-of-a-string-literal

In [86]:
sen = s[0].encode('utf-8')
print (type(s[0]), s[0], type(sen), sen, ' decoded: ', sen.decode('utf-8'))
print (sen[0], sen[1], sen[2])
ss = [236, 150, 180]
print (ss)
css = ''.encode('utf-8')
#css = css + 0xec
print (css)
bss = bytes(ss)
print (bss, bss.decode('utf-8'))

<class 'str'> 어 <class 'bytes'> b'\xec\x96\xb4'  decoded:  어
236 150 180
[236, 150, 180]
b''
b'\xec\x96\xb4' 어
